In [21]:
 import json
import requests
import string
from collections import Counter
import time


ACCESSTOKEN  = 'EAACEdEose0cBAFJbfxm8FU9fX2hkJ6R4KUXgSRYkDdLYRuZAX61JlTRi7FMa5sq5x1YMLlnJzxCmSGRPNgLOahgxTfrTrcVZAel4GMFJSiYtPg31LJRqZB1vzI1uPbnIBpDJWejZAFpjww0ZAKRX7aE1M8H4PnZCqLRv5ki9tQJgZDZD'
PostUrl ='https://graph.facebook.com/v2.5/15925638948/posts?limit=100&fields=attachments,message,created_time,shares&access_token='+ACCESSTOKEN

res = requests.get(PostUrl)

jd = json.loads(res.text)


PostID= jd['data'][0]['id']
print jd['data'][1]['shares']['count']
len(jd['data'])


6


100

In [22]:
def comment(id):
    ComentUrl ='https://graph.facebook.com/v2.5/'+id+'/comments?fields=comments.limit(200)&limit=200&access_token='+ACCESSTOKEN
    ComentRes = requests.get(ComentUrl)
    ComentJd = json.loads(ComentRes.text)
    
    ComentCount=0
    ReplyComent=0
#     try:
#         ComentJd['paging']
#     except:
#         return 0
    
    while 'next' in ComentJd['paging']:
       
        ComentCount += len(ComentJd['data'])
        
        for data in ComentJd['data']:
            
            if 'comments' in data:
               
                ReplyComent+=len(data['comments']['data'])
                
        
        ComentUrl =ComentJd['paging']['next']
        ComentRes = requests.get(ComentUrl)
        ComentJd = json.loads(ComentRes.text)

    ComentCount += len(ComentJd['data'])
    for data in ComentJd['data']:
        
        if 'comments' in data:

            ReplyComent+=len(data['comments']['data'])

    return ComentCount,ReplyComent
    
   
a=comment('15925638948_10154270907828949')    
print a[0],a[1]

179 79


In [23]:
def like(id):
    likesUrl ='https://graph.facebook.com/v2.5/'+id+'/likes?limit=1000&access_token='+ACCESSTOKEN
    likesRes = requests.get(likesUrl)
    likesJd = json.loads(likesRes.text)
    likeCount=0
    try:
        likesJd['paging']
    except:
        return 0
    while 'next' in likesJd['paging']:
        likeCount+= len(likesJd['data'])
        likesUrl =likesJd['paging']['next']
        likesRes = requests.get(likesUrl)
        likesJd = json.loads(likesRes.text)
        
    likeCount+= len(likesJd['data'])    
    return  likeCount
like('15925638948_10154712143383949')    

4960

In [24]:
def insertSQL():
    
    command=('INSERT INTO Finace ([NO],[Year],[MovieName],[LikeCount],[ShareCount],[CommentCount],[ReplyCount],[TotalGross]'
      ',[AttatchmentTitle]'
     ' ,[PostMessage]'
      ',[id]) VALUES (?,?,?,?,?,?,?,?,?,?,?)')

    cursor.execute(command,(len(my_dict),year,name[2],LikeCount,ShareCount,commentCount,replyCount,Gross,attatchTitle,Postmessage,id))
    conn.commit()
    


# 主程式區

In [25]:
from collections import defaultdict
my_dict = defaultdict(int)


print len(my_dict)
import pyodbc
conn = pyodbc.connect('DRIVER={SQL Server};SERVER=163.14.72.42;DATABASE=FB_NewCrawler;UID=sa;PWD=scucc')
cursor = conn.cursor()

#粉絲
PostUrl ='https://graph.facebook.com/v2.5/15925638948?fields=likes&access_token='+ACCESSTOKEN
res = requests.get(PostUrl)
jd = json.loads(res.text)
Fans= jd['likes']

PostUrl ='https://graph.facebook.com/v2.5/15925638948/posts?fields=attachments,message,created_time,shares&access_token='+ACCESSTOKEN
res = requests.get(PostUrl)
jd = json.loads(res.text)
count=0
end=0

while( 'next' in jd['paging'] ):
    for data in jd['data']:

        
        post=data
        attatchTitle=""
        attatchMessage=""
        commentCount=0
        replyCount=0
        LikeCount=0
        ShareCount=0
        Postmessage=""
        id=data['id']
       
        
        
        if (data['id']==jd['data'][-1]['id']):
            Url=jd['paging']['next']

            Res = requests.get(Url)
            jd = json.loads(Res.text)
            
            
        
        #例外
        if('message'in post):
            Postmessage=post['message']
        if('shares'in post):
            ShareCount=post['shares']['count']
        #attachments
        if('attachments' in post):
            if('title' in post['attachments']['data'][0]):
                attatchTitle=post['attachments']['data'][0]['title']
            if('description' in post['attachments']['data'][0]):
                attatchMessage=post['attachments']['data'][0]['description']
            if( 'trailer'not in attatchTitle.lower()  ):
                continue
            year= data['created_time'][:4]
            cursor.execute("select * from [BoxOffice] where year=? order by id,year",year)
            Postrows = cursor.fetchall()
            for name in Postrows:
                if  name[2] in attatchTitle:
                    if len(name[2])<=3:
                        continue
                    if name[2] in my_dict:
                        my_dict[name[2]] += 1
                    else:
                        print id
                        my_dict[name[2]] = 1
                        CommentReply=comment(id)
                        Gross=name[3]
                        LikeCount=like(id)
                        commentCount=CommentReply[0]
                        replyCount=CommentReply[1]
                        
                        print LikeCount,ShareCount,commentCount,replyCount,Gross
                        insertSQL()
                        
                        
#                     print name[2]
#                     print attatchTitle
#                     count+=1
#                     print len(my_dict)
#                     print count 
                    
                    break
        else:
            continue
    try:
        jd['paging']
    except:
        print end
        break
        
        

        

0
15925638948_10154602894258949
1492 404 276 165 2403419
15925638948_10154594092413949
6227 2679 341 180 803301
15925638948_10154580772198949
2882 725 152 52 108215432
15925638948_10154532538363949
652 266 28 5 64200
15925638948_10154527207533949
2665 937 275 141 41668494
15925638948_10154475678603949
120 12 5 3 112680839
15925638948_10154438254293949
931 318 122 39 42816119
15925638948_10154417727558949
2190 811 201 94 20969350
15925638948_10154409010008949
1945 1133 228 106 36053483
15925638948_10154403929373949
653 877 43 21 583664
15925638948_10154347497548949
415 203 30 15 1339152
15925638948_10154342751873949
382 101 38 21 15436808
15925638948_10154337552653949
1884 604 212 177 160215
15925638948_10154331725958949
634 144 60 28 158526005
15925638948_10154317871128949
991 590 127 64 55451779
15925638948_10154299907598949
132 18 17 13 55124043
15925638948_10154289661218949
535 228 44 13 82051601
15925638948_10154276485133949
74 28 6 1 3220371
15925638948_10154276235783949
1960 892 

In [10]:
cursor.close()
conn.close()


In [24]:

import pyodbc
conn = pyodbc.connect('DRIVER={SQL Server};SERVER=163.14.72.42;DATABASE=FB_NewCrawler;UID=sa;PWD=scucc')
cursor = conn.cursor()

In [12]:

import pyodbc
conn = pyodbc.connect('DRIVER={SQL Server};SERVER=163.14.72.42;DATABASE=FB_NewCrawler;UID=sa;PWD=scucc')
cursor = conn.cursor()
year='2016'
cursor.execute("select * from [BoxOffice] where year=? order by id,year",year)
Postrows = cursor.fetchall()
for name in Postrows:
    newName=name[2].split('(')
    
    command="update BoxOffice set [MovieName]=? where MovieName=? "

    cursor.execute(command,(newName[0],name[2]))
    conn.commit()

In [17]:

year='2016'
cursor.execute("select * from [BoxOffice] where year=? order by id,year",year)
Postrows = cursor.fetchall()
for name in Postrows:
    print name[2]

Finding Dory
Captain America: Civil War
The Secret Life of Pets
The Jungle Book 
Deadpool
Zootopia
Batman v Superman: Dawn of Justice
Suicide Squad
Jason Bourne
Star Trek Beyond
X-Men: Apocalypse
Kung Fu Panda 3
Ghostbusters 
Central Intelligence
The Legend of Tarzan
Bad Moms
Sully
The Angry Birds Movie
Independence Day: Resurgence
The Conjuring 2
Sausage Party
Ride Along 2
Don't Breathe
Teenage Mutant Ninja Turtles: Out of the Shadows
The Purge: Election Year
Alice Through the Looking Glass
Pete's Dragon 
10 Cloverfield Lane
Lights Out
The Magnificent Seven 
The Divergent Series: Allegiant
Now You See Me 2
Ice Age: Collision Course
The Boss
London Has Fallen
Miracles from Heaven
My Big Fat Greek Wedding 2
Me Before You
The BFG
Neighbors 2: Sorority Rising
The Shallows
Barbershop: The Next Cut
13 Hours: The Secret Soldiers of Benghazi
The Huntsman: Winter's War
Warcraft
Kubo and the Two Strings
How to Be Single
Mike and Dave Need Wedding Dates
War Dogs
Storks
Money Monster
Nerve
Risen


In [ ]:
from collections import defaultdict

my_dict = defaultdict(int)

key='a'

if key in my_dict:
    my_dict[key] += 1
else:
    my_dict[key] = 1
print len(my_dict)

In [13]:
a='fan'
print len(a)

3
